In [ ]:
#Importando as bibliotecas necessárias
import pandas as pd
import psycopg2 as psy
import os
from dotenv import load_dotenv

try:

    load_dotenv() # - carregando as variáveis de ambiente para que possam ser utilizadas
    
    conexao = psy.connect(
        dbname = os.getenv("DB_NAME"),
        user = os.getenv("USER"),
        password = os.getenv("PASSWORD"),
        host = os.getenv("HOST"),
        port = os.getenv("PORT")
    ) # - estabelecendo a conexão com o banco de dados

    cursor = conexao.cursor()

    #Execução do comando para trazer os dados da tabela original no banco de dados
    cursor.execute(
        '''SELECT * FROM super_adm;'''
    )

    #Criando um dataframe apartir dos dados capturados
    df = pd.DataFrame(cursor.fetchall(), columns=[item[0] for item in cursor.description])
    display(df) # - mostrando o dataframe


    cursor.close() # - fechando conexão
    conexao.close() # - fechando conexão

except Exception as e:
    print(f"Erro ao conectar com o banco: "+str(e)) # - tratando exceção, (caso ocorra)

,id,nome,cargo,email,senha_hash
0,1,Carlos Silva,Super Administrador,carlos.silva@avicola.com.br,Admin12345


In [ ]:
#Realizando as pseudonimizaçãoes necessárias
#Criando um clone do dataframe original
df_super_pseudo = df.copy()

#pseudonimização da coluna de id
df_super_pseudo['id'] = [ '*' * (len(str(id))-1)+str(id)[-1] if len(str(id))>1 else "*" for id in df_super_pseudo['id']]
#pseudonimização da coluna de nome
df_super_pseudo['nome'] = df_super_pseudo['nome'].apply(lambda nome: nome[:3]+f'{"*" * (len(nome)-3)}')
#pseudonimização da coluna de cargo
df_super_pseudo['cargo'] = df_super_pseudo['cargo'].apply(lambda cargo: cargo[:3]+f'{"*" * (len(cargo)-3)}')
#pseudonimização da coluna de email
df_super_pseudo['email'] = df_super_pseudo['email'].apply(lambda email: email[:2] + f"{'*' * (email.find('@') -2)}"+email[email.find('@') +1:])
#pseudonimização da coluna de senha_hash - hashing


In [35]:
#Renomeando as colunas do dataframe
df_super_pseudo.rename(columns={
    "id":"id",
    "nome":"nome_pseudo",
    "cargo" : "cargo_pseudo",
    "email": "email_pseudo",
    "senha_hash": "senha_hash"
}, inplace=True)

In [ ]:
#Transferindo todas as informações do dataframe pseudonimizado para o banco de dados 


try:
    
    conexao = psy.connect(
        dbname = os.getenv("DB_PSEUDO"),
        user = os.getenv("USER"),
        password = os.getenv("PASSWORD"),
        host = os.getenv("HOST"),
        port = os.getenv("PORT")
    ) # - estabelecendo a conexão com o banco de dados

    cursor = conexao.cursor()

    #execução do comando para trazer os dados da tabela original
    query = """
    INSERT INTO super_adm_pseudo (id, nome_pseudo, cargo_pseudo, email_pseudo, senha_hash)
    VALUES (%s, %s, %s, %s, %s)
    ON CONFLICT (id) 
    DO UPDATE SET
        nome_pseudo = EXCLUDED.nome_pseudo,
        cargo_pseudo = EXCLUDED.cargo_pseudo,
        email_pseudo = EXCLUDED.email_pseudo
        
    """ # - comando que vai ser aplicado

    valores = [tuple(x) for x in df_super_pseudo.to_numpy()] # - transformando cada linha do dataframe em uma tupla

    for c in valores:
        cursor.execute(query, c) # - executando o comando no banco de dados

    conexao.commit() # - salvando as alterações


    cursor.close() # - fechando conexão 
    conexao.close() # - fechando conexão 

except Exception as e:
    print(f"Erro ao conectar com o banco: "+str(e)) # - tratando exceção, (caso ocorra)

In [37]:
#Tranferindo as informações originais e pseudonimizadas para arquivos csv
df.to_csv('originais_superadm.csv')
df_super_pseudo.to_csv('pseudo_superadm.csv')